In [1]:
# 🧪 Step 2.7: Positive + Negative 구조 포함 CSV 병합 및 인코딩
import pandas as pd
import numpy as np
import os

# 경로 설정
POS_PATH = "../data/processed/positive_structures.csv"
NEG_PATH = "../data/processed/negative_structures.csv"
X_PATH = "../data/features/X_all.npy"
Y_PATH = "../data/features/y_all.npy"

# 데이터 로드
pos = pd.read_csv(POS_PATH)
neg = pd.read_csv(NEG_PATH)

# 병합
df = pd.concat([pos, neg], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # 셔플

# 인코딩 매핑
seq_map = {'A':0, 'C':1, 'G':2, 'U':3}
str_map = {'(':0, ')':1, '.':2}

def encode(seq, struct):
    mat = np.zeros((len(seq), 7))
    for i in range(len(seq)):
        mat[i, seq_map.get(seq[i], 0)] = 1
        mat[i, 4 + str_map.get(struct[i], 2)] = 1
    return mat

# 전체 인코딩
X = np.stack([encode(seq, dot) for seq, dot in zip(df["sequence"], df["structure"])])
y = df["label"].values.astype(np.float32)

# 저장
os.makedirs("../data/features", exist_ok=True)
np.save(X_PATH, X)
np.save(Y_PATH, y)

print(f"✅ 저장 완료: X_all.npy (shape={X.shape}), y_all.npy (shape={y.shape})")

✅ 저장 완료: X_all.npy (shape=(40188, 51, 7)), y_all.npy (shape=(40188,))


## 🧬 Step 2.7: Positive + Negative 통합 및 One-hot Encoding

이 단계에서는 구조 정보가 포함된 Positive / Negative CSV 파일을 병합하고,  
sequence + structure 정보를 One-hot encoding 하여 `.npy`로 저장합니다.

### 📌 인코딩 방식
- 서열(A/C/G/U) → 4채널
- 구조(dot-bracket) → 3채널 (`(`, `)`, `.`)
- 총 7채널로 구성된 `(51, 7)` 텐서

### 💾 출력
- `X_all.npy`: 모델 입력 텐서
- `y_all.npy`: 이진 라벨 벡터